In [42]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import pandas as pd

In [43]:
scaler = StandardScaler()
df = pd.read_excel('/Foodwatch_input_data.xlsx')
df[['Crop_Cover_Percentage', 'Normalized_Food_Price', 'GDP_Per_Capita', 'Pests', 'Transport_Network_Density', 'Food_Storage_Groceries', 'Conflict_Per_Capita', 'Food_Wastage_Per_Capita']] = scaler.fit_transform(df[['Crop_Cover_Percentage', 'Normalized_Food_Price', 'GDP_Per_Capita', 'Pests', 'Transport_Network_Density', 'Food_Storage_Groceries', 'Conflict_Per_Capita', 'Food_Wastage_Per_Capita']])
two_factors = df.loc[:, ['Food_Wastage_Per_Capita', 'Conflict_Per_Capita', 'Region', 'IPC_Level']]



In [44]:
#Splitting the Data
X = two_factors.drop(['Region', 'IPC_Level'], axis=1)
y = two_factors['IPC_Level']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [45]:
# Create PyTorch Dataset and DataLoader
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X.values, dtype=torch.float32)
        self.y = torch.tensor(y.values, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [46]:
train_dataset = CustomDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [47]:
# Define the Neural Network Architecture
class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [38]:
model = NeuralNetwork(input_size=X_train.shape[1])

In [39]:
# Define Loss Function and Optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [40]:
# Training the Model
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), targets)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

Epoch [1/100], Loss: 4.268941879272461
Epoch [2/100], Loss: 0.6259691119194031
Epoch [3/100], Loss: 0.8757810592651367
Epoch [4/100], Loss: 0.5550104379653931
Epoch [5/100], Loss: 0.3604367673397064
Epoch [6/100], Loss: 0.35836511850357056
Epoch [7/100], Loss: 0.23980361223220825
Epoch [8/100], Loss: 0.3150872588157654
Epoch [9/100], Loss: 0.3395639955997467
Epoch [10/100], Loss: 0.19104652106761932
Epoch [11/100], Loss: 0.19297951459884644
Epoch [12/100], Loss: 0.3198612928390503
Epoch [13/100], Loss: 0.30448293685913086
Epoch [14/100], Loss: 0.26447200775146484
Epoch [15/100], Loss: 0.16132698953151703
Epoch [16/100], Loss: 0.21708062291145325
Epoch [17/100], Loss: 0.1927681863307953
Epoch [18/100], Loss: 0.2632829248905182
Epoch [19/100], Loss: 0.37921321392059326
Epoch [20/100], Loss: 0.2321217805147171
Epoch [21/100], Loss: 0.2070484459400177
Epoch [22/100], Loss: 0.11587405204772949
Epoch [23/100], Loss: 0.2609114944934845
Epoch [24/100], Loss: 0.27991124987602234
Epoch [25/100],

In [41]:
# Evaluation
model.eval()
with torch.no_grad():
    test_inputs = torch.tensor(X_test.values, dtype=torch.float32)
    predictions = model(test_inputs).squeeze()

In [49]:
mse = mean_squared_error(y_test, predictions.numpy())
print(mse)

0.2781167201397615
